# Deep Learning Assignment 1

In [138]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import seaborn as sns
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import time

To left justify the tables in markdown so they are nicer to read

In [141]:
%%html
<style>
table {float:left}
</style>

## Logistic Regression implementation.
The implementation of logistic regression here is as described in the lecture notes. The weights are created as a vector, each uniformly distributed on $[0,1]$. The 'back propogation' step takes place in the stochastic_gradient_descent function. 

In [118]:
class LogisticRegression:
    def __init__(self, n_inputs, sgd_thresh = 10e-8, sgd_max_iters = 2500, alpha = 0.0001):
        self.n_inputs = n_inputs
        self.weights = np.random.rand(self.n_inputs, 1).astype('f')
        #print(self.weights)
        self.bias = 1.0
        self.alpha = alpha
        self.sgd_thresh = sgd_thresh
        self.sgd_max_iters = sgd_max_iters
        
    def sigmoid(self, x):
        return 1/(1+np.exp(-1*x))
    
    def predict(self, x):
        x = np.array(x).reshape(-1,1)
        return int(np.round(self.sigmoid(np.dot(self.weights.T, x) + self.bias)))
    
    def log_loss(self, X, Y):
        #dot = [np.dot(self.weights.T, x) for x in X]
        
        y_hats = [self.sigmoid(np.dot(self.weights.T, x) + self.bias) for x in X]
        self.yhats = y_hats
        #print(np.sum(y_hats))
        J_w_b = (-1/len(Y))*np.sum([y*np.log(y_hat)+(1-y)*np.log(1-y_hat) for y,y_hat in zip(Y,y_hats)])
        return J_w_b

    def stochastic_gradient_descent(self,X,Y):
        X = np.array(X)
        Y = np.array(Y)
        J = self.log_loss(X,Y)
        for i in range(self.sgd_max_iters):
            idx = np.random.randint(len(X))
            x_sample, y_sample = X[idx], Y[idx]
            diff = self.predict(x_sample) - y_sample

            self.weights = self.weights - (self.alpha*diff*x_sample).reshape(-1,1)

            self.bias = self.bias - self.alpha*diff

            
            if self.log_loss(X,Y) - J < self.sgd_thresh:
                break
            #print(self.log_loss(X,Y), J)
            J = self.log_loss(X,Y)
            
    def score(self, x_test, y_test):
        x_test = np.array(x_test)
        y_test = np.array(y_test)
        correct = 0
        total = len(x_test)
        for x,y in zip(x_test, y_test):
            if self.predict(x) ==  y:
                correct += 1
        return correct/total

## Performance on the blobs250 dataset

We observe then on the easiest of all the data sets that logistic regression perfectly splits the datasets and scores 100% on the test set.

In [120]:
data = pd.read_csv(path+"blobs250.csv")

x_train, x_test, y_train, y_test = train_test_split(data.drop(['Class'], axis = 1), data['Class']) 

lr = LogisticRegression(x_train.shape[1],sgd_max_iters=200,sgd_thresh=-np.inf,alpha=10e-2)
lr.stochastic_gradient_descent(x_train,y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
lr.score(x_test, y_test)

1.0

## Performance on the moons400 dataset

The data is loaded in using pandas and is split into training and test sets using the train_test_split function from the model_selection module in the sci-kit learn package. On the harder of the two problems it achieves an accuracy of 84%

In [124]:
path = "/home/oisin/MAI_work/ongoing_assignments/DeepLearning/data/"

data = pd.read_csv(path+"moons400.csv")

x_train, x_test, y_train, y_test = train_test_split(data.drop(['Class'], axis = 1), data['Class']) 

lr = LogisticRegression(x_train.shape[1],sgd_max_iters=200,sgd_thresh=-np.inf,alpha=10e-2)
lr.stochastic_gradient_descent(x_train,y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
lr.score(x_test, y_test)

0.84

## Shallow Neural Network Performance
Below is the code for the implementation of the neural network. Since one of my enhancements was to implement deep neural networks (support for multiple layers), the code is identical to the enhancement code. This is simply because there was no point implementing code specific to one hidden layer and then implementing code for many layers when just generalising first does the trick. 

### Layer Class
This implementation has a Layer class, where the weights and layer activation function are stored. The weights are initially $\mathcal{N}(0,1)$ as are the biases. The class implements "fwdprop_ouput" which calculates a and z for the Layer, as seen in the notes.

### Network Class
The Network class creates the layers, as directed by the inputs. These inputs are in the form of 2 lists, m and n say, where the $i^{th}$ member of m is the number of nodes in layer $i$, and the $i^{th}$ member of n is the activation function class for that layer (see below).

### Activation Function Class
The activation functions are created with an "f" method and a "deriv" method. These represent the function itself and the derivative of the function, to be used in the back propagation step, as required by the chain rule

In [205]:
class Layer:
    
    def __init__(self, prev_no_nodes, no_nodes, activation_function, is_input=False):
        if not(is_input):
            bound = np.sqrt(6)/np.sqrt(prev_no_nodes + no_nodes)
            self.weights = np.random.uniform(-1*bound,bound,size = (prev_no_nodes,no_nodes))
            self.bias = np.random.normal(-1*bound,bound, size = (no_nodes,1))
            self.act_f = activation_function()
            
        else:
            self.act_f = activation_function()
        self.no_nodes = no_nodes
        self.is_input = is_input

    def fwdprop_output(self, X):
        if self.is_input:
            self.a = X
            self.z = self.a
            return X
        X = X.reshape(-1,1)
        self.z = np.dot(self.weights.T,X) + self.bias
        self.a = self.act_f.f(self.z)
        return self.a
    
    
    
class IdentityActivation:
    def __init__(self):
        pass
    
    def f(self,x):
        return x
    
    def deriv(self, x):
        return x
    
    
class ReluActivation:
    def __init__(self):
        pass
    
    def f(self, x):
        return np.maximum(x,0)
        
    def deriv(self, x):
        x[x>0] = 1
        x[x<0] = 0
        return x
        
class SigmoidActivation:
    def __init__(self):
        pass
    
    def f(self,x):
        x=x.astype('f')
        return 1/(1+np.exp(-x))
    
    def deriv(self, x):
        return self.f(x)*(1-self.f(x))

class Network:
    
    def __init__(self, no_nodes_layer, activation_function, loss_function = "log_loss", lamb = 0):
        '''
        TODO: Sort out the fact that the first layer doesn't have an activation function (DOESNT FUCKIN NEED ONE AHAHHA)
        TODO: Add the loss_functions
        TODO: Finish train and predict/score
        '''
        self.no_nodes_layer = no_nodes_layer
        self.activation_function = activation_function
        self.input_size = no_nodes_layer[0]
        self.no_layers = len(self.no_nodes_layer)
        self.lamb = lamb
        self.no_params = sum([self.no_nodes_layer[i]*self.no_nodes_layer[i-1] for i in range(1,len(self.no_nodes_layer))])
        
        if isinstance(no_nodes_layer, list):
            '''
            TODO: add functionality so that differnet layers can have different activation functions 
            '''
            assert(self.no_nodes_layer[-1] == 1 or self.no_nodes_layer[-1] == 2)
            self.layers = [Layer(self.no_nodes_layer[i-1],self.no_nodes_layer[i],activation_function[i]) for i in range(1,len(self.no_nodes_layer))]
            input_layer = Layer(0,no_nodes=self.no_nodes_layer[0],activation_function=activation_function[0], is_input=True)
            self.layers = [input_layer] + self.layers

        else:
            #Come up with a better default
            self.layers = [Layer(no_nodes_layer,1)]

        self.W = np.zeros(0)
        #turn all the weight matrices into one long weight vector so one can find the l_p norm of it.
        self.W = np.concatenate([self.W] +[layer.weights.flatten() for layer in self.layers[1:]])
        #print(np.linalg.norm(self.W, 2))
    

    
    def log_loss(self,X,Y):
        y_hats = [self.fwdpropagate(x) for x in X]
        J_w_b = (-1)*sum([y*np.log(y_hat)+(1-y)*np.log(1-y_hat) for y,y_hat in zip(Y,y_hats)])
        J_w_b += self.lamb*np.linalg.norm(self.W, 2)
        return J_w_b
        
        
    def fwdpropagate(self, _input):
        
        if len(_input) != self.layers[0].no_nodes:
            print(f"Input must be of length {self.layers[0].no_nodes}, it is of length {len(_input)}")
        
        self.a_s = []   
        a = _input

        
        for layer in self.layers:
            a = layer.fwdprop_output(a)
            self.a_s.append(a)
        
        #self.layer_outputs = []
        y_hat = a
        #self.layer_outputs.append(y_hat)
        
        self.prediction = int(np.round(y_hat))


    def predict(self, x_test):
        self.fwdpropagate(x_test)
        return self.prediction

    def stochastic_gradient_descent(self, x_train, y_train, alpha = 0.001, n_iter = 10, val_split = 0, verbose = 0):
        '''
        Implementation of stochastic gradient descent with regularization. 
        '''
        
        
        x_train = np.array(x_train)
        y_train = np.array(y_train)
        
        if val_split:
            ind = int(val_split*len(x_train))
            x_val = x_train[:ind]
            y_val = y_train[:ind]
            x_train = x_train[ind:]
            y_train = y_train[ind:]
        
        #combine the x and ys into one array to make suffling more straightforward
        comb = np.c_[x_train.reshape(len(x_train), -1), y_train.reshape(len(y_train), -1)]
        x_train_c = comb[:, :x_train.size//len(x_train)].reshape(x_train.shape)
        y_train_c = comb[:, x_train.size//len(x_train):].reshape(y_train.shape)
        #keep track of total training time and print initial accuracy sans training.
        start = time.time()
        ita = score = self.score(x_train,y_train)
        print(f"Training on {len(x_train)} samples for {n_iter} epochs on {self.no_layers} layers with {self.no_params} parameters...")
        print(f"Initial training accuracy: {ita:.4f}%...")
        for epoch in range(1,n_iter+1):
            if verbose: print(f"Running epoch {epoch} of {n_iter}")
            
            #This is just a fancy way of shuffling the x and ys together.
            x_train_c = comb[:, :x_train.size//len(x_train)].reshape(x_train.shape)
            y_train_c = comb[:, x_train.size//len(x_train):].reshape(y_train.shape)

            np.random.shuffle(comb)
        
            for x,y in zip(x_train_c, y_train_c):
                self.backward_propagate_error(x,y)
                for i,layer in enumerate(self.layers):
                    if layer.is_input:
                        continue
                    else:
                        layer.weights = layer.weights - (alpha*np.array(self.delta_w[i])) #.reshape(layer.weights.shape)
                        layer.bias = layer.bias - alpha*np.array(self.delta_b[i]) #.reshape(layer.bias.shape)
            if val_split:
                val_score = self.score(x_val, y_val)
            
            score = self.score(x_train,y_train)
            if verbose: print(f"Training accuracy: {score:.4f}%")
            if val_split and verbose: print(f"Validation Accuracy: {val_score:.4f}%")
            #-----------------
        end = time.time()  
            
        score = self.score(x_train,y_train)
        val_score = self.score(x_val, y_val)
        print(f"Training accuracy: {score:.4f}%")
        
        if val_split: 
            print(f"Validation Accuracy: {val_score:.4f}%")
            
        print(f"Ran {n_iter} epochs in {end-start:.2f} seconds")
    
    
    def score(self, x_test, y_test):
        '''
        Simple score function that gets the accuracy for a given test set
        '''
        x_test = np.array(x_test)
        y_test = np.array(y_test)
        correct = 0
        total = len(x_test)
        for x,y in zip(x_test, y_test):
            self.fwdpropagate(x)
            if self.predict(x) ==  y:
                correct += 1
        return correct/total
    
    
    
    def backward_propagate_error(self, x, y):
        ''' The backward propogate error function is based on the algorithm from http://cs229.stanford.edu/notes2020spring/cs229-notes-deep_learning.pdf
            (on the 2nd last page). Note the difference in derivatives of cost function from these notes, as 
            the notes use 1/2 squared error as the cost function.
            
            The function fisrt does the forward propagtaion to ensure the z's and a's of each layer exist. Then,
            using the chain rule, we compute the gradients for each weight in each layer.
        '''
        self.fwdpropagate(x)
        delta = [None]*self.no_layers 
        delta_w = [None]*self.no_layers
        delta_b = [None]*self.no_layers
   
        for i in range(self.no_layers - 1, -1, -1):
            if i == self.no_layers - 1:
                if y == 1:
                    q = (y/(self.layers[i].a+ 10e-7))
                else:
                    q = -1*((1-y)/(1-(self.layers[i].a- 10e-7)))
                delta[i] = -1*q*self.layers[i].act_f.deriv(self.layers[i].z)
            else:
                delta[i] = ((self.layers[i+1].weights)@(delta[i+1]))*self.layers[i].act_f.deriv(self.layers[i].z).reshape(-1,1)
                delta_w[i+1] = ((delta[i+1])@(self.layers[i].a.reshape(1,-1))).T + 2*self.lamb*self.layers[i+1].weights
                delta_b[i+1] = delta[i+1]

        self.delta_w = delta_w
        self.delta_b = delta_b
        self.delta = delta

## Shallow Neural Network on blobs250 and moons400
To create the shallow neural network we must create a list with the number of notes in the input layer, the number in the hidden layer, and the number in the output layer, which will always be 1. It is neccessary to create a list of the same length with the activation function in the respective layers.  
  
  Note: I do the validation split after the train-test split. So to produce a .70-.15-.15 split I first split .85-.15 train-test on the original data, then a 14/17-3/17 train-validation split on the .85

In [175]:
path = "/home/oisin/MAI_work/ongoing_assignments/DeepLearning/data/"

data = pd.read_csv(path+"blobs250.csv")

x_train, x_test, y_train, y_test = train_test_split(data.drop(['Class'], axis = 1), data['Class'], test_size = 0.15) 

acts = [IdentityActivation, SigmoidActivation, SigmoidActivation]


In [176]:
net = Network([3, 25, 1],activation_function=acts, lamb = 0)
net.stochastic_gradient_descent(x_train, y_train, alpha=0.01, n_iter=10, val_split=3/17);

Training on 175 samples for 10 epochs on 3 layers with 100 parameters...
Initial training accuracy: 0.7943%...
Training accuracy: 1.0000%
Validation Accuracy: 1.0000%
Ran 10 epochs in 0.30 seconds


In [177]:
net.score(x_test,y_test)

1.0

In [188]:
data = pd.read_csv(path+"moons400.csv")

x_train, x_test, y_train, y_test = train_test_split(data.drop(['Class'], axis = 1), data['Class'], test_size=0.15) 

acts = [IdentityActivation, SigmoidActivation, SigmoidActivation]

net = Network([2, 25, 1],activation_function=acts, lamb = 0)

net.stochastic_gradient_descent(x_train, y_train, alpha=0.01, n_iter=50, val_split=3/17)

net.score(x_test, y_test)

Training on 280 samples for 50 epochs on 3 layers with 75 parameters...
Initial training accuracy: 0.4893%...
Training accuracy: 0.8643%
Validation Accuracy: 0.8667%
Ran 50 epochs in 2.18 seconds


0.95

## Results
There is naturally no improvement in the easier dataset as the logistic regression had 100% test accracy, as did the shallow net. The shallow net improved on the harder of the two data sets by about 13%.

|       | Logistic Regression | Shallow Neural Network |
|-------|---------------------|------------------------|
| Blobs | 100%                | 100%                   |
| Moons | 84%                 | 95%                    |

## Loading in the cifar-10 image data
Below are the functions used to load in the CIFAR-10 data, convert it to greyscale, and split into the training and test sets. Unpickle is supplied by the University of Toronto.

In [192]:
###--- Functions to get data and to modify ---###

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        d = pickle.load(fo, encoding='bytes')
    return d

def greyscale_data(data):
    '''
    FORMULA IS L = R * 299/1000 + G * 587/1000 + B * 114/1000
    where R,G,B are, obviously the RGB colour values.

    In the data we have a vector of length 3*1024. Split this into a matrix of size 3x1024 
    then convert using the formula
    '''
    data_2 = []
    for i,x in enumerate(data):
        r = x[:1024]
        g = x[1024:2048]
        b = x[2048:]
        grey = ((r * 299/1000) + (g * 587/1000) + (b * 114/1000))/255
        data_2.append(grey)
    return np.array(data_2)


path_to_images = "/home/oisin/MAI_work/ongoing_assignments/DeepLearning/cifar-10-batches-py/"
# datas = [unpickle(path_to_images+"data_batch_"+str(i)) for i in range(1,6)]
datas = [unpickle(path_to_images+"data_batch_1")]
x = [d[b'data'] for d in datas]
y = [d[b'labels'] for d in datas]
x_ = np.concatenate(x)
y_ = np.concatenate(y)
data =  [[imput_x, imput_y - 2] for imput_x, imput_y in zip(x_,y_) if imput_y == 2 or imput_y == 3]

_all_data = np.array(data)
x_data = greyscale_data(_all_data[:,0])
y_data = _all_data[:,1]
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.15)
x_train, x_test, y_train, y_test = x_train.astype('f'), x_test.astype('f'), y_train.astype('f'), y_test.astype('f')
inp_size = len(x_train[0])

<ipython-input-192-7d0abaed5133>:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  _all_data = np.array(data)


### Shallow Network on CIFAR-10 data (Cats and Birds)
We create a neural network with 1000 nodes in 1 hidden layer, each layer with the sigmoid activation function. The first layer doesn't need an activation function, the IdentityActivation doesn't actually do anything, it is more for illustrative puposes

In [196]:
acts = [IdentityActivation, SigmoidActivation, SigmoidActivation]
net = Network([x_train.shape[1], 1000, 1],activation_function=acts, lamb = 0)

In [197]:
net.stochastic_gradient_descent(x_train,y_train, alpha=0.005, n_iter=100, val_split=3/17)

Training on 1433 samples for 100 epochs on 3 layers with 1025000 parameters...
Initial training accuracy: 0.5010%...
Training accuracy: 0.8618%
Validation Accuracy: 0.5505%
Ran 100 epochs in 2323.57 seconds


In [198]:
net.score(x_test,y_test)

0.4902597402597403

We can see clearly that the shallow net is not expressive enough to describe the image data. 

## Deep Learning Enhancement
As mentioned before, the main enhancement made was the ability to add more hidden layers, but a few extra tweaks were added also. When observing the training of the neural network with one hidden layer there were to major issues that stood out initially. 
* Firstly, the training was extreamly sensitive to the inital weight values (with respect to the scale I was usuing). A solution is offered by Glorot and Bengio in http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf where they suggest that the weights be distributed as: $$\mathcal{U}\left[-\frac{\sqrt{6}}{\sqrt{m+n}},\frac{\sqrt{6}}{\sqrt{m+n}}\right] $$ 
where m is the number of inputs to a layer and n is the number of nodes (or outputs) in the layer.

* Secondly, the network was extreamly prone to overfitting. To overcome this I introduced $L_2$ regularisation. The implementation came straight from the notes from class.

So we can see that for a network with 1 hidden layer with 1000 nodes we reach about 62% training accuracy and 54.5% accuracy on the validation set and the test set. This leads us to believe that the network is overfitting on the training data. To combat this we can introduce more layers and a regularisation parameter $\lambda$. This further adaption allowed the training accuracy to reach 100% and the test accuracy to reach 64%. While still overfitting we have a significant improvement in test accuracy, so the network has generalised somewhat when compared to the single hidden layer without regularisation. 

In [199]:
acts = [IdentityActivation, ReluActivation,ReluActivation,ReluActivation,ReluActivation,ReluActivation,SigmoidActivation]
net = Network([x_train.shape[1],1000,500,500, 250, 100, 1],activation_function=acts, lamb = 0.001)

In [200]:
net.stochastic_gradient_descent(x_train, y_train, alpha=0.001, n_iter = 100, val_split= 0.11)

Training on 1549 samples for 100 epochs on 7 layers with 1924100 parameters...
Initial training accuracy: 0.4984%...
Training accuracy: 1.0000%
Validation Accuracy: 0.6440%
Ran 100 epochs in 3954.16 seconds


In [201]:
net.score(x_test, y_test)

0.6038961038961039

### Observations on testing
